In [7]:
pip install jq

Note: you may need to restart the kernel to use updated packages.Collecting jq
   ---------------------------------------- 0.0/416.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/416.9 kB ? eta -:--:--
    --------------------------------------- 10.2/416.9 kB ? eta -:--:--
    --------------------------------------- 10.2/416.9 kB ? eta -:--:--
    --------------------------------------- 10.2/416.9 kB ? eta -:--:--
   -- ------------------------------------ 30.7/416.9 kB 186.2 kB/s eta 0:00:03
   --- ----------------------------------- 41.0/416.9 kB 163.4 kB/s eta 0:00:03
   ----- --------------------------------- 61.4/416.9 kB 217.9 kB/s eta 0:00:02
   ------- ------------------------------- 81.9/416.9 kB 269.5 kB/s eta 0:00:02
   ---------- --------------------------- 112.6/416.9 kB 327.2 kB/s eta 0:00:01
   ------------- ------------------------ 143.4/416.9 kB 355.0 kB/s eta 0:00:01
   --------------- ---------------------- 174.1/416.9 kB 388.2 kB/s eta 0:00:01
   


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import os
from dotenv import load_dotenv
import json
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# Load the scraped data from a JSON file
json_file_path = "scraped_recipes.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

text_documents = [entry["content"] for entry in json_data if "content" in entry]


In [11]:
# Define jq_schema
jq_schema = ".[]"

# Load JSON data from a file
json_loader = JSONLoader(file_path="scraped_recipes.json", 
                         jq_schema=jq_schema, 
                         text_content=False)
docs = json_loader.load()

In [14]:
print("Loaded documents:", len(docs))
print(docs[:3])

Loaded documents: 1715
[Document(metadata={'source': 'C:\\Users\\USER\\Desktop\\LangChian Course\\LangChain-RAG Course\\recipe-generator\\scraped_recipes.json', 'seq_num': 1}, page_content='{"title": "Creamy Vegan White Bean Chili", "content": "Minimalist Baker Disclosure: This post may contain affiliate links which provide us a small commission when used for purchase. We\'re grateful for your support! We love aclassic (tomato-based) chili, but a CREAMY chili? It\\u2019s hard to say no to! Mix up your chili rotation with this creamy vegan white bean chili with spicy green chiles, sweet corn, and nutrient-packed spinach or kale. It\\u2019s a cozy, subtly spiced, nourishing meal that comes together in just30 minutes. Let us show you how it\\u2019s done! This EASY vegan white bean chili starts with asimple yet flavorfulbase of saut\\u00e9ed onion, garlic, cumin, cayenne, salt, and pepper. Next, we add green chiles, white beans, and corn. The green chiles add lots of flavor, thewhite beans

In [15]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs[:100])
vectordb = FAISS.from_documents(documents,OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B3067FDC90>, search_kwargs={})

In [19]:
## Query search on the FAISS Vector database
query = "What are the best approach in preparing Banana Chocolate Pecan Muffins"
result = vectordb.similarity_search(query)
result[0].page_content

'as I try to avoid consuming too much sugar. I only used 1 Tbsp. pure organic maple syrup, instead of the recommended 1/2 c. coconut sugar, & they were plenty sweet enough due to the medium ripe bananas, so 1 Tbsp. maple syrup worked out perfectly! :) Also I used tapioca starch instead of potato starch, and the texture was absolutely perfect, using flax eggs & added 1/4 c almond flour. Didn\\u2019t use walnuts due to allergies. This recipe calls for 10-12 muffins, mine was only 7 muffins, and baking time of 25-30 minutes was enough, anything more than that they would\\u2019ve been too dark. I added this recipe to my favorites, & will definitely be making it again. Delicious recipe like always! Thank you, & looking forward to many more! Support @ Minimalist Bakersays February 5, 2024 at 1:47 pm We\\u2019re so glad you enjoyed them, Melissa! Thanks so much for the lovely review and for sharing your modifications. xo Cheryl Renocksays February 1, 2024 at 3:44 pm I want to make these howev

In [21]:
# Create a retriever tool for searching recipes
retriever_tool = create_retriever_tool(
    retriever,
    "recipe_search",
    "Search for recipes based on ingredients, cuisine, or dietary preferences. Use this tool to find relevant recipes!"
)

retriever_tool

Tool(name='recipe_search', description='Search for recipes based on ingredients, cuisine, or dietary preferences. Use this tool to find relevant recipes!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B306B88180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B3067FDC90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001B306C46840>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B3067FDC90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables

In [23]:
# Load the wikipedia tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [44]:
# Combine the tools
tools = [retriever_tool, wiki]

# Instantiate an llm
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Promt
prompt = ChatPromptTemplate.from_template(
    """
    You are a smart recipe assistant. Provide answers to recipe-related questions based only on the given context.
    Make sure to offer detailed and step-by-step guidance to help the user.

    <context>
    {context}
    </context>

    Question: {input}

    Your Answer:
    """
)


In [46]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [47]:
## Create a retrieval chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "How can I prepare Garlic & Herb Breaded Delicata Squash?"})

In [48]:
response['answer']

'To prepare Garlic & Herb Breaded Delicata Squash, follow these steps:\n\n1. Preheat your oven to 400°F (200°C).\n2. In a bowl, mix together 1 cup of panko breadcrumbs, 2 tablespoons of nutritional yeast, 1 teaspoon of dried rosemary, 1 teaspoon of garlic powder, and a pinch of sea salt. Optionally, you can add some fresh parsley for color.\n3. Slice your delicata squash into 1/2-inch thick pieces.\n4. Dip each squash slice into a bowl of olive oil to help the breading stick.\n5. Press the oiled squash slices into the breadcrumb mixture, coating them on all sides.\n6. Place the coated squash slices on a baking sheet lined with parchment paper.\n7. Bake in the preheated oven for about 15 minutes or until the breading is golden brown and the squash is tender.\n8. Once done, remove from the oven and serve your Garlic & Herb Breaded Delicata Squash as a crispy, tender, herby, garlicky, and flavorful side dish.\n\nEnjoy your delicious roasted delicata squash!'